In [ ]:
pip install pandas


In [ ]:
pip install sklearn

In [6]:
pip install numpy

Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install tensorflow

In [ ]:
pip install keras

In [ ]:
pip install pickle

In [ ]:
pip install music21

In [23]:
import numpy
import pickle
from music21 import converter, instrument, note, chord
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense 
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import BatchNormalization
from keras.layers import Activation

In [21]:
def generate():
    with open('data/notes', 'rb') as filepath:
        notes = pickle.load(filepath)
    
    pitchnames = sorted(set(item for item in notes))
    n_vocab = len(set(notes))
    
    network_input, normalized_input = prep_seq(notes, pitchnames, n_vocab)
    model = network_create(normalized_input, n_vocab)
    prediction_output = note_generate(model, network_input, pitchnames, n_vocab)
    midi_create(prediction_output)

In [22]:
def prep_seq(array, ptchname, n_vocab):
    notetoint = dict((no, number) for number, no in enumerate(ptchname))
    
    seq_len = 100
    net_input = []
    out = []
    for i in range(0, len(array) - seq_len, 1):
        seq_in = array[i:i + seq_len]
        seq_out = array[i + seq_len]
        net_input.append([notetoint[char] for char in seq_in])
        out.append(notetoint[seq_out])
    
    n_patterns = len(net_input)
    
    norm_input = numpy.reshape(net_input, (n_patterns, seq_len, 1))
    # normalize input
    norm_input = norm_input / float(n_vocab)
    
    return (net_input, norm_input)

In [24]:
def network_create(net_in, voc):
    m = Sequential()
    m.add(LSTM(
        512,
        input_shape=(net_in.shape[1], net_in.shape[2]),
        recurrent_dropout=0.3,
        return_sequences=True
    ))
    m.add(LSTM(512, return_sequences=True, recurrent_dropout=0.3))
    m.add(LSTM(512))
    m.add(BatchNormalization())
    m.add(Dropout(0.3))
    m.add(Dense(256))
    m.add(Activation('relu'))
    m.add(BatchNormalization())
    m.add(Dropout(0.3))
    m.add(Dense(voc))
    m.add(Activation('softmax'))
    m.compile(loss='categorical_crossentropy', optimizer='rmsprop')
    
    m.load_weights('weights.hdf5')
    
    return m

In [25]:
def note_generate(m, net_in, ptchname, voc):
    s = numpy.random.randint(0, len(net_in) - 1)
    
    inttonote = dict((num, n) for num, n in enumerate(ptchname))
    
    pat = net_in[s]
    pred_out = []
    
    for notin in range(500):
        pred_in = numpy.reshape(pat, (1, len(pat), 1))
        pred_in = pred_in / float(voc)
        
        pred = m.predict(pred_in, verbose=0)
        
        i = numpy.argmax(pred)
        res = inttonote[i]
        pred_out.append(res)
        
        pat.append(i)
        pat = pat[1:len(pat)]
    
    return pred_out

In [29]:
from music21 import converter, instrument, note, chord, stream
def midi_create(pred_out):
    out_not = []

    for pat in pred_out:
        if ('.' in pat) or pat.isdigit():
            noteinchord = pat.split('.')
            notes = []  # Create an empty list for storing the notes in a chord
            for cur_note in noteinchord:
                newnote = note.Note(int(cur_note))  # Use `note.Note` instead of `array.Note`
                newnote.storedInstrument = instrument.Piano()
                notes.append(newnote)
            newchord = chord.Chord(notes)  # Create a chord from the list of notes
            newchord.offset = 10
            out_not.append(newchord)

        else:
            newnote = note.Note(pat)  # Use `note.Note` instead of `array.Note`
            newnote.offset = 10
            newnote.storedInstrument = instrument.Piano()
            out_not.append(newnote)

    midistream = stream.Stream(out_not)

    midistream.write('midi', fp='output.mid')

In [30]:
if __name__ == '__main__':
    generate()